In [1]:
import simpy
import numpy as np

In [2]:
RANDOM_SEED = 42
#number of id check queues
NUM_ID_CHECK_SERVERS = 5
#number of body scanners/queues
NUM_BODY_SCANNERS = 5
T_INTER = 0.2 # mean passenger inter_arrival time (poisson distributed)
ID_CHECK_TIME = 0.75  # mean time each server takes to check an ID (exponentially distributed)
SCAN_TIME_LOW, SCAN_TIME_HIGH = 0.5, 1  # time taken at a scanner to scan a passenger (uniformly distributed)
SIM_TIME = 120     # Simulation time in minutes

In [3]:
class IdCheck:
    """An ID check queue at the airport has a limited number of servers working in parallel.
    Passengers get on a queue and get their IDs checked within an exponential processing time
    with mean of 0.75 minutes.
    
    """
    def __init__(self, env, num_servers):
        self.env = env
        self.server = simpy.Resource(env, num_servers)
    
        
    def check_passenger_id(self, passenger):
        """The id check process."""
        yield self.env.timeout(np.random.exponential(1/ID_CHECK_TIME))
        print(f"{passenger}'s ID check completed")
        
class PersonalCheck:
    """Personal Check Queues with a limited number of personal scanners working in parallel. 
    Passengers get on the shortest queue after the ID check and when it's their turn, are processing
    at time that is uniformly distributed between 0.5 and 1 minute.
    
    """
    def __init__(self, env, num_scanners):
        self.env = env
        self.scanner = simpy.Resource(env, num_scanners)
        
    def scan_passenger(self, passenger):
        """The id check process."""
        yield self.env.timeout(np.random.uniform(SCAN_TIME_LOW, SCAN_TIME_HIGH))
        print(f"{passenger.name} has completed personal check")

class Passenger:
    """A passenger who arrives at airport security."""
    
    def __init__(self, env, name, idchecker, bodyscanner):
        self.env = env
        self.name = name
        self.idchecker = idchecker
        self.bodyscanner = bodyscanner
        
    def get_id_checked(self, env, idchecker):
        # id check
        print(f'{self.name} arrives at security check at {float(self.env.now):.2f}')
        with self.idchecker.server.request() as request:
            yield request
        
        print(f'{self.name} starts id check at {float(self.env.now):.2f}')
        yield self.env.process(self.idchecker.check_passenger_id(passenger))
        
        print(f'{self.name} completes id check at {float(env.now):.2f}')
        
def passenger(env, name, idchecker, bodyscanner):
    """The passenger process. Each passenger arriving at security check joins an 
    id check queue and waits their turn.
    
    After completing Id check, the passenger then joins the shortest personal check queue,
    completes personal scan, and proceeds to the boarding area.
    
    """
    # id check
    print(f'{name} arrives at security check at {float(env.now):.2f}')
    with idchecker.server.request() as request:
        yield request
        
        print(f'{name} starts id check at {float(env.now):.2f}')
        yield env.process(idchecker.check_passenger_id(passenger))
        
        print(f'{name} completes id check at {float(env.now):.2f}')
    
    #personal check
    print(f'{name} joins personal scanner queue at {float(env.now):.2f}')
    with bodyscanner.scanner.request() as request:
        yield request
        
        print(f'{name} starts personal scan at {float(env.now):.2f}')
        yield env.process(bodyscanner.scan_passenger(passenger))
        
        print(f'{name} completes personal check at {float(env.now):.2f}')
        
def setup(env, num_id_checkers, num_personal_scanners, t_inter):
    """Create a id check resource with a number of server, personal check resources,
    and a number of initial passengers at security. 
    Keep adding passengers at the inter-arrival rate
    """
    
    #create the IDcheck resource
    idchecker = IdCheck(env, num_id_checkers)
    
    bodyscanner = PersonalCheck(env, num_personal_scanners)
    
    # create ten initial passengers
    for i in range(10):
        env.process(passenger(env, f'Passenger {i}', idchecker, bodyscanner))

        
    # Create more cars while the simulation is running
    while True:
        yield env.timeout(np.random.poisson(t_inter))
        i += 1
        env.process(passenger(env, f'Passenger {i}', idchecker, bodyscanner))

In [4]:
print('Airport Security')
np.random.seed(RANDOM_SEED)

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_ID_CHECK_SERVERS, NUM_BODY_SCANNERS, T_INTER))

#Execute
env.run(until=SIM_TIME)

Airport Security
Passenger 0 arrives at security check at 0.00
Passenger 1 arrives at security check at 0.00
Passenger 2 arrives at security check at 0.00
Passenger 3 arrives at security check at 0.00
Passenger 4 arrives at security check at 0.00
Passenger 5 arrives at security check at 0.00
Passenger 6 arrives at security check at 0.00
Passenger 7 arrives at security check at 0.00
Passenger 8 arrives at security check at 0.00
Passenger 9 arrives at security check at 0.00
Passenger 10 arrives at security check at 0.00
Passenger 0 starts id check at 0.00
Passenger 1 starts id check at 0.00
Passenger 2 starts id check at 0.00
Passenger 3 starts id check at 0.00
Passenger 4 starts id check at 0.00
<function passenger at 0x0000020BF3ACB288>'s ID check completed
Passenger 3 completes id check at 0.08
Passenger 3 joins personal scanner queue at 0.08
Passenger 3 starts personal scan at 0.08
Passenger 5 starts id check at 0.08
<function passenger at 0x0000020BF3ACB288>'s ID check completed
Pas

AttributeError: 'function' object has no attribute 'name'